In [ ]:
import zmq #pip install pyzmq
import json
from typing import NamedTuple
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt
from scipy.fft import fft, fftfreq
SAMPLING_RATE = 64 #Hz

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.setsockopt(zmq.SUBSCRIBE, b'bvp')
socket.connect("tcp://192.168.1.179:5050")


In [ ]:
bvp = []
# while True:
for _ in range(2048):
    #  Wait for next request from client
    # msg = json.loads(socket.recv())
    # sample = Sample(**msg)
    # print(f'Received {sample=}')
    msg = json.loads(socket.recv().decode().split(' ')[1])
    bvp.append(float(msg['bvp']))
    print(f'Received {msg=}')

In [ ]:
bvp = np.array(bvp)
bvp = bvp.clip(-1.7,1.7)

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def apply_lowpass_filter(data, cutoff_freq, fs, order=5):
    b, a = butter_lowpass(cutoff_freq, fs, order=order)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Applying low-pass filter
cutoff_freq = 3  # Cutoff frequency in Hz
filtered_data = apply_lowpass_filter(bvp, cutoff_freq, SAMPLING_RATE)

# Plotting original and filtered signals
import matplotlib.pyplot as plt

plt.figure()
# plt.plot(bvp, 'b-', label='Original Signal')
plt.plot(filtered_data, 'r-', label='Filtered Signal')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Low-Pass Filtered Signal at 128 Hz Cutoff')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# FFT
n = len(bvp)
frequencies = fftfreq(n, 1/SAMPLING_RATE)
fft_values = fft(bvp)
plt.plot(frequencies[:n//2], np.abs(fft_values[:n//2]))
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude')
plt.title('FFT of Original Signal')
plt.grid()

In [ ]:
frequencies[:n//2][np.abs(fft_values[:n//2]).argmax()]